In [1]:
!pip install selenium

In [2]:
!pip install webdriver_manager

Сайт GeekJob имеет особенности:

1. Нет фильтрации по дате, только сортировка. Если задать специальность, выставить сортировку и взять ссылку, то сортировка по такой ссылке не работает. Поэтому используем селениум (заполняем строку поиска, выбираем сортировку по дате, нажимаем кнопку поиска).
2. Если поиск по вакансии не дал результатов, на сайте все равно отображаются вакансии, нерелевантные поиску. Поэтому перед тем, как парсить, проверяем кол-во найденных вакансий (если там пустое значение, то не парсим).
3. Уровень (Джуниор, Стажер) можно посмотреть уже на страничке вакансии, поэтому по уровню фильтруем уже после перехода на страничку.
4. На сайте обычно мало актуальных вакансий, поэтому данные выбираются за текущую и предыдущую дату.


In [3]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import random
import numpy as np
import pandas as pd
import time

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('enable-automation')
options.add_argument('--disable-infobars')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-browser-side-navigation')
options.add_argument("--remote-debugging-port=9222")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")

In [5]:
from datetime import date, timedelta
from selenium.webdriver import Keys
import csv
import logging
from os import path, remove  

#  Функция для получения текущей и предыдущей даты в виде число-месяц как на сайте
def rename_date(date):
    date_dict = {"01": "января", "02": "февраля", "03": "марта",
                 "04": "апреля", "05": "мая", "06": "июня",
                 "07": "июля", "08": "августа", "09": "сентября", 
                 "10": "октября", "11": "ноября", "12": "декабря"}
    list_date = date.split()
    return list_date[0].lstrip('0') + ' ' + date_dict.get(list_date[1])

#  Получаем текущую дату в виде строки как на сайте
current_date = rename_date(date.strftime(date.today(), '%d %m'))
prev_date = rename_date(date.strftime(date.today() - timedelta(days=1), '%d %m'))

#  В словаре date_dict ключ - дата в формате как на сайте для поиска вакансии,
#  а значение - datetime, для записи результата в файл
date_dict = {current_date: date.today(), prev_date:date.today() - timedelta(days=1)}
date_dict

{'19 апреля': datetime.date(2023, 4, 19),
 '18 апреля': datetime.date(2023, 4, 18)}

In [6]:
#  Инициализируем переменные

URL = 'https://geekjob.ru/'

#  Список строк запроса для выбора по специальностям
query_list = ['аналитик', 'analyst', 'дата сайентист', 'data scientyst', 'DS']  

#  В название csv-файла включаем текущую дату
csv_file = f"geekjob_{date.strftime(date.today(),'%d%m%Y')}.csv" 
log_file = f"geekjob_log_{date.strftime(date.today(),'%d%m%Y')}.log"

#  Список уровней вакансий 
level_list = ['Джуниор', 'Стажер']

# Словарь тегов для парсинга
tags_dict = {'vacancy': '//a[@href="/vacancies"]',
             'queryinput': '//input[@name="queryinput"]',
             'radio': '//section[@class="col s12 m4"][2]',
             'find_button': '//section[@class="col s12 m12"]/button[@class="btn btn-small waves-effect"]',
             'vacancies_num': '//div[@id="serp"]',
             'items': '//li[@class="collection-item avatar"]',
             'dates': '//p[@class="truncate datetime-info"]',
             'vacancy_hrefs': '//p[@class="truncate vacancy-name"]/a[1]',
             'element_present': '//article[@class="row vacancy"]',
             'level': '//b[contains(text(), "Уровень должности")]/following-sibling::a',
             'info': '//div[@id="vacancy-description"]',
             'title': '//h1',
             'company': '//h5[@class="company-name"]/a',
             'location': '//div[@class="location"]',
             'skills': '//b[contains(text(), "Отрасль и сфера применения")]/preceding-sibling::a',
             'salary': '//span[@class="salary"]',
             'jobformat': '//span[@class="jobformat"]',
             'company_field': '//b[contains(text(), "Отрасль и сфера применения")]/following-sibling::a'}


In [7]:
# Удаляем существующий файл лога, если он есть, чтобы создавать новый файл во время каждого выполнения  
if path.isfile(log_file):  
    remove(log_file)  

#  Задаем параметры логгирования
logging.basicConfig(
    level=logging.INFO,
    filename = log_file,
    format = "%(asctime)s - %(module)s - %(levelname)s - %(funcName)s: %(lineno)d - %(message)s",
    datefmt='%H:%M:%S',
    )

s = Service('C:\\Users\\Компьютер\\Desktop\\Системная\\chromedriver.exe')
driver = webdriver.Chrome(service=s)
#driver = webdriver.Chrome(options=options)
driver.get(URL)
driver.maximize_window()

with open(csv_file, 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file, delimiter=';')

    #  Записываем заголовки таблицы в файл
    writer.writerow(['title', 'company', 'country', 'location', 'salary', 
                     'source', 'link', 'date', 'company_field', 'description',
                     'skills', 'job_type'])

        #  Создаем список для контроля уникальности обработанных вакансий
    find_list = [] 
    vacancy_index = 0
    
    for query in query_list:
        print(f'\nПарсим вакансии по запросу "{query}"')
        logging.info(f'Поиск вакансий по запросу "{query}"')
        vacancy = driver.find_element(By.XPATH, tags_dict.get('vacancy'))
        driver.execute_script("arguments[0].click();", vacancy)
        queryinput = driver.find_element(By.XPATH, tags_dict.get('queryinput')).send_keys(query)
        time.sleep(1)
        radio = driver.find_element(By.XPATH, tags_dict.get('radio')).click()
        time.sleep(1)
        find_button = driver.find_element(By.XPATH, tags_dict.get('find_button')).click()
        time.sleep(5)
        vacancies_num = driver.find_element(By.XPATH, tags_dict.get('vacancies_num')).text
        
        if vacancies_num != '':
            logging.info(vacancies_num)
            logging.info('Парсим из них только вакансии нужного уровня с отбором по дате:')
            main_window = driver.window_handles[0]
            count = 1
            items = driver.find_elements(By.XPATH, tags_dict.get('items'))
            dates = driver.find_elements(By.XPATH, tags_dict.get('dates'))
            vacancy_hrefs = driver.find_elements(By.XPATH, tags_dict.get('vacancy_hrefs'))
            
            for data, vacancy_href, item in zip(dates, vacancy_hrefs, items):
                link = vacancy_href.get_attribute('href')
                if data.text in date_dict and link not in find_list:
                    date = data.text                    
                    time.sleep(3)
                    driver.execute_script("arguments[0].click();", vacancy_href)
                    time.sleep(3)
                    window_after = driver.window_handles[1]
                    driver.switch_to.window(window_after)
                    
                    #  Ждем когда загрузятся данные по вакансии
                    element_present = EC.presence_of_element_located((By.XPATH, 
                                                                      tags_dict.get('element_present')))
                    WebDriverWait(driver, 5).until(element_present) 
                    
                    #  Парсим уровень специальности
                    try:
                        level = driver.find_elements(By.XPATH, 
                                                     tags_dict.get('level'))
                        level = ', '.join(elem.text for elem in level)
                    except Exception as ex:
                        level = None
                        logging.error(f"Ошибка при парсинге level: {ex}")

                    #  Если в уровень вакансии входит хотя бы один элемент списка level_list, 
                    #  парсим остальную информацию и записываем ее в файл
                    if not (level is None) and any(el if el in level else None for el in level_list):

                        try:
                            info = driver.find_element(By.XPATH, 
                                                       tags_dict.get('info'))
                            description = BeautifulSoup(info.get_attribute('innerHTML')).text
                        except Exception as ex:
                            description = None
                            logging.error(f"Ошибка при парсинге description: {ex}")

                        try:
                            title = driver.find_element(By.XPATH, 
                                                        tags_dict.get('title')).text
                        except Exception as ex:
                            title = None
                            logging.error(f"Ошибка при парсинге title: {ex}")
                            
                        try:
                            company = driver.find_element(By.XPATH, 
                                                          tags_dict.get('company')).text
                        except Exception as ex: 
                            company = None
                            logging.error(f"Ошибка при парсинге company: {ex}")

                        try:
                            location = driver.find_element(By.XPATH, 
                                                           tags_dict.get('location')).text.split(',')
                            country = location[-1].strip() if len(location)>0 else '' 
                            location = ', '.join(location[:-1])
                        except Exception as ex:
                            country = None
                            location = None
                            logging.error(f"Ошибка при парсинге location: {ex}")

                        try:
                            salary = driver.find_element(By.XPATH, 
                                                         tags_dict.get('salary')).text
                        except Exception as ex:
                            salary = None
                            logging.error(f"Ошибка при парсинге salary: {ex}")

                        try:
                            jobformat = driver.find_element(By.XPATH, 
                                                            tags_dict.get('jobformat')).text.split('\n')
                            experience = jobformat[1]
                            job_type = ', '.join(jobformat[0].split(' • '))
                        except Exception as ex:
                            experience = None
                            job_type = None  
                            logging.error(f"Ошибка при парсинге job_type: {ex}")

                        try:
                            company_field = driver.find_elements(By.XPATH, 
                                                                 tags_dict.get('company_field'))
                            company_field = ', '.join(el.text for el in company_field)
                            company_field = company_field.rstrip(level)
                        except Exception as ex:
                            company_field = None
                            logging.error(f"Ошибка при парсинге company_field: {ex}")
                                                        
                        try:
                            skills = driver.find_element(By.XPATH, 
                                                         tags_dict.get('skills')).text
                            skills = ', '.join(skills.split(' • '))
                        except:
                            skills = None
                            logging.error(f"Ошибка при парсинге skills: {ex}")

                        # Записываем собранные данные в файл
                        
                        row = [title, company, country, location, salary, 
                               'GeekJob', link, date_dict.get(date),
                               company_field, description, skills, job_type]
                        row = [el if el else None for el in row]
                        
                        writer.writerow(row) 

                        find_list.append(link)
                        print(count, end = ' ')
                        logging.info(f"Записали в файл вакансию с индексом {vacancy_index}")
                        count += 1
                        vacancy_index += 1
                        
                    driver.close()
                    driver.switch_to.window(main_window)

                    vacancy_href.send_keys(Keys.DOWN)
            
driver.close()
print(f'\nПарсинг закончен, результат помещен в файл {csv_file}')
logging.info(f'Парсинг закончен, результат помещен в файл {csv_file}')


Парсим вакансии по запросу "аналитик"
1 2 
Парсим вакансии по запросу "analyst"

Парсим вакансии по запросу "дата сайентист"

Парсим вакансии по запросу "data scientyst"
1 
Парсим вакансии по запросу "DS"

Парсинг закончен, результат помещен в файл geekjob_19042023.csv


In [8]:
df = pd.read_csv(csv_file, sep=';', keep_default_na = False )
df

,title,company,country,location,salary,source,link,date,company_field,description,skills,job_type
0,Аналитик бизнес-процессов (настройка и внедрен...,amgroup автоматизация,Россия,Ростов-на-Дону,от 50 000 до 100 000 ₽,GeekJob,https://geekjob.ru/vacancy/643e90d8c2ed3f28120...,2023-04-18,"CRM, SaaS/PaaS, Заказная разработка, Консалтин...",amgroup автоматизация – аккредитованная IT-ком...,"Аналитика, Data Science, Big Data","Удаленная работа, Работа в офисе"
1,Пишущий редактор в Сириус.Курсы,Sirius.online,Россия,Москва,от 80 000 ₽,GeekJob,https://geekjob.ru/vacancy/643e586f4c77d6adac0...,2023-04-18,Образование,О нас: Мы создаём продвинутые онлайн-курсы для...,Контент,Работа в офисе
2,Разработчик C#,Рекрутер Анна,Россия,Иваново,от 64 000 ₽,GeekJob,https://geekjob.ru/vacancy/643eb74ee6d35befda0...,2023-04-18,Заказная разработка,Компания НТТ занимается разработкой CRM-систем...,Информационные технологии,Удаленная работа


In [9]:
with open(log_file, 'r') as file:
    content = file.read()
    print(content)

00:00:18 - 42081754 - INFO - <cell line: 19>: 33 - Поиск вакансий по запросу "аналитик"
00:00:26 - 42081754 - INFO - <cell line: 19>: 45 - Найдено 202 вакансии
00:00:26 - 42081754 - INFO - <cell line: 19>: 46 - Парсим из них только вакансии нужного уровня с отбором по дате:
00:00:58 - 42081754 - INFO - <cell line: 19>: 158 - Записали в файл вакансию с индексом 0
00:01:30 - 42081754 - INFO - <cell line: 19>: 158 - Записали в файл вакансию с индексом 1
00:01:39 - 42081754 - INFO - <cell line: 19>: 33 - Поиск вакансий по запросу "analyst"
00:01:47 - 42081754 - INFO - <cell line: 19>: 45 - Найдено 23 вакансии
00:01:47 - 42081754 - INFO - <cell line: 19>: 46 - Парсим из них только вакансии нужного уровня с отбором по дате:
00:01:47 - 42081754 - INFO - <cell line: 19>: 33 - Поиск вакансий по запросу "дата сайентист"
00:01:55 - 42081754 - INFO - <cell line: 19>: 33 - Поиск вакансий по запросу "data scientyst"
00:02:02 - 42081754 - INFO - <cell line: 19>: 45 - Найдено 102 вакансии
00:02:02 - 4